In [17]:
import pandas as pd
#df = pd.read_csv(r'/home/refu0917/lungcancer/seerdb/csv_folder/seer_reg13.csv')
df = pd.read_csv(r'/home/refu0917/lungcancer/seerdb/csv_folder/seer_reg17.csv',index_col=[0])
rename_dict = {'Patient ID':'id',
            'Site recode ICD-O-3/WHO 2008':'site',
            'Sequence number':'seq_number',
            'Sex' : 'Gender',
            'Year of diagnosis' : 'diag_year',
            'Age recode with <1 year olds':'Age',
            'CS tumor size (2004-2015)':'TMRSZ',
            'Regional nodes positive (1988+)':'LYMND',
            'Derived AJCC Stage Group, 7th ed (2010-2015)':'AJCCstage',
            'Visceral and Parietal Pleural Invasion Recode (2010+)':'SSF2',
            'Separate Tumor Nodules Ipsilateral Lung Recode (2010+)':'SSF1',
            'Grade (thru 2017)':'DIFF',
            'Survival months':'survival_month',
            'Vital status recode (study cutoff used)': 'vital_sts',
            'ICD-O-3 Hist/behav, malignant': 'histologic',
            'Behavior code ICD-O-3' : 'behavior',
            'Regional nodes examined (1988+)' : 'LYMND_exm',
            'Primary Site - labeled':'primary_site',
            'Survival months flag': 'survival_month_flag'}
df = df.rename(columns=rename_dict)

df['histologic'] = df['histologic'].apply(lambda x:x[:6])
df['primary_site'] = df['primary_site'].apply(lambda x:x[:5])
df = df[df['survival_month'] !='Unknown']
df['survival_month'] = df['survival_month'].astype('int')
df = df.drop(columns=['Patient ID.1'])
print(len(df))

7351789


In [9]:
df.head()

,id,seq_number,primary_site,Age,histologic,Gender,diag_year,AJCCstage,DIFF,TMRSZ,SSF2,SSF1,LYMND,survival_month,survival_month_flag
0,21,2nd of 2 or more primaries,C62.9,40-44 years,9061/3,Male,2002,Blank(s),Unknown,Blank(s),Blank(s),Blank(s),98,208,1
1,24,2nd of 2 or more primaries,C20.9,65-69 years,8140/3,Male,2000,Blank(s),Well differentiated; Grade I,Blank(s),Blank(s),Blank(s),0,49,1
2,30,2nd of 2 or more primaries,C19.9,35-39 years,8140/3,Female,2008,Blank(s),Moderately differentiated; Grade II,035,Blank(s),Blank(s),1,143,1
3,43,2nd of 2 or more primaries,C38.1,60-64 years,8801/3,Female,2008,Blank(s),Well differentiated; Grade I,090,Blank(s),Blank(s),98,0,1
4,52,2nd of 2 or more primaries,C44.4,85+ years,8742/3,Male,2008,Blank(s),Unknown,999,Blank(s),Blank(s),98,26,1


In [18]:
ipc = df[(df.seq_number == 'One primary only') | (df.seq_number == '1st of 2 or more primaries')]
spm = df[df.seq_number == '2nd of 2 or more primaries']
#ipc = ipc[(ipc.diag_year <=2010) & (ipc.diag_year >=2004)]
#spm = spm[spm.diag_year >= 2004]
print(len(ipc), len(spm))

ipc = ipc.loc[ipc['histologic'].isin(['8012/3', '8046/3', '8070/3', '8140/3', '8240/3', '8250/3', '8560/3', '9053/3'])]
print(len(ipc))
ipc = ipc.loc[ipc['primary_site'].isin(['C33.9', 'C34.0', 'C34.1', 'C34.2', 'C34.3', 'C34.8' ,'C34.9'])]
print(len(ipc))
ipc = ipc[ipc['survival_month_flag'] == 1]
print(len(ipc))


6201005 1150784
2338834
467793
448100


In [19]:
'''ipc_flwup = ipc[ipc['survival_month'] !='Unknown']
ipc_flwup['survival_month'] = ipc_flwup['survival_month'].astype('int')
ipc_flwup = ipc_flwup[ipc_flwup['survival_month']>=48]
print(len(ipc_flwup))
'''
#spm = spm[spm['survival_month'] !='Unknown']
#spm['survival_month'] = spm['survival_month'].astype('int')
spm = spm[spm.id.isin(ipc.id.tolist())]
spm = spm[spm.survival_month >=6]
print(len(spm))

22952


In [20]:
rule1 = spm.loc[~spm['primary_site'].isin(['C33.9', 'C34.0', 'C34.1', 'C34.2', 'C34.3', 'C34.8' ,'C34.9'])]
print(len(rule1))
spm = spm.loc[spm['primary_site'].isin(['C33.9', 'C34.0', 'C34.1', 'C34.2', 'C34.3', 'C34.8' ,'C34.9'])]
print(len(spm))

12036
10916


In [22]:
spm['comp_histologic'] = ipc.loc[ipc.id.isin(spm.id.tolist())]['histologic'].tolist()
spm['comp_diagyear'] = ipc.loc[ipc.id.isin(spm.id.tolist())]['diag_year'].tolist()

rule2 = spm.loc[(spm['histologic']!=spm['comp_histologic'])]
spm = spm.loc[(spm['histologic']==spm['comp_histologic'])]
print(len(rule2), len(spm))
rule3 = spm.loc[(spm['diag_year']-spm['comp_diagyear'])>=2]
spm = spm.loc[(spm['diag_year']-spm['comp_diagyear']<2)]
print(len(rule3), len(spm))

6630 4286
2478 1808


In [28]:
rule2 = rule2.drop(columns=['comp_histologic'])
rule3 = rule3.drop(columns=['comp_diagyear'])
spc = pd.concat([rule1, rule2, rule3], axis = 0)
ipc['Class'] = 0
ipc.loc[ipc[ipc.id.isin(spc.id.tolist())].index,"Class"] = 1
ipc.Class.value_counts()

In [58]:
ipc['Age'] = ipc['Age'].apply(lambda x:int(x.split('-')[0][0]) if x[:2] != '85' else int(8))
ipc = ipc.loc[~ipc['TMRSZ'].isin([888, 990, 996, 997, 998])]
ipc = ipc.loc[~ipc['LYMND'].isin([95])]
ipc = ipc.loc[~ipc['AJCCstage'].isin(['OCCULT'])]

In [60]:
def regional_node_encode(x):
    if x == 'Blank(s)':
        return 999
    elif x == 0 or x == 98:
        return 1
    elif x == 99 or x == 97:
        return 9
    elif 1<=x<=2:
        return 2
    elif 3<=x<=6:
        return 3
    elif 7<=x<=15:
        return 4
    elif 16<=x:
        return 5

def tumor_encode(size):
    if size == 'Blank(s)':
        return 999
    size = int(size)
    if size == 999:
        return 9
    elif size <= 49:
        return 1
    elif 50 <= size <= 99:
        return 2
    elif 100 <= size <= 149:
        return 3
    else:
        return 4

invasion_map_dict = {'PL0; No evidence; Tumor does not completely traverse the elastic layer of pleura':1,
                    'Tumor extends to pleura, NOS; not stated if visceral or parietal':2,
                    'PL1 or PL2; Invasion of visceral pleura present, NOS':2,
                    'PL3; Tumor invades into or through the parietal pleura OR chest wall':2,
                    'Not documented; No resection of primary; Not assessed or unknown if assessed':9,
                    'Blank(s)':999}

septumor_map_dict = {'None; No intrapulmonary mets; Foci in situ/minimally invasive adenocarcinoma':1,
                    'Separate nodules of same hist type in ipsilateral lung, same lobe':2,
                    'Separate nodules of same hist type in ipsilateral lung, different lobe':2,
                    'Separate tumor nodules, ipsilateral lung, unknown if same or different lobe':2,
                    'Separate nodules of same hist type in ipsilateral lung, same AND different lobes':2,
                    'Not documented; Primary tumor is in situ; Not assessed or unknown if assessed':9,
                    'Blank(s)':999}

grade_map_dict = {'Moderately differentiated; Grade II':2,
                  'Poorly differentiated; Grade III':3,          
                  'Well differentiated; Grade I':1,             
                  'Undifferentiated; anaplastic; Grade IV':4,
                  'Unknown':9,
                  'Blank(s)':999}
gender_dict = {'Male':1, 'Female':2}

ipc['Gender'] = ipc['Gender'].map(gender_dict)
ipc['LYMND'] = ipc['LYMND'].map(regional_node_encode)
ipc['TMRSZ'] = ipc['TMRSZ'].map(tumor_encode)
ipc['SSF2'] = ipc['SSF2'].map(invasion_map_dict)
ipc['SSF1'] = ipc['SSF1'].map(septumor_map_dict)
ipc['DIFF'] = ipc['DIFF'].map(grade_map_dict)
ipc['AJCCstage'] = ipc['AJCCstage'].replace('UNK Stage',9)
ipc['AJCCstage'] = ipc['AJCCstage'].replace('Blank(s)',999)

for i,j in zip(['V','III', 'II','I'], [4,3,2,1]):
    index = ipc[ipc['AJCCstage'].str.contains(i,na=False)].index
    ipc.loc[index,'AJCCstage'] = ipc[ipc['AJCCstage'].str.contains(i, na=False)]['AJCCstage'].apply(lambda x:j)

ipc = ipc.fillna(999)
ipc = ipc.drop(columns=['id','seq_number', 'primary_site', 'histologic', 'survival_month', 'survival_month_flag', 'diag_year'])

In [76]:
ipc_no_null = ipc[ipc!=999]
ipc_no_null = ipc_no_null.dropna()
ipc_no_null.Class.value_counts()
ipc_no_null['LOC'] = 9

In [87]:
ipc_no_null.to_csv(r'/home/refu0917/lungcancer/data/seerdb.csv')

In [68]:
from tableone import TableOne, load_dataset
#drawe table one

columns = ["Class","Gender", "Age",
        "AJCCstage", "DIFF", "LYMND", "TMRSZ",
        "SSF1", "SSF2"]
groupby = ["Class"]
categorical = ["Class","Gender", "Age",
        "AJCCstage", "DIFF", "LYMND", "TMRSZ",
        "SSF1", "SSF2"]
seer_tableone = TableOne(ipc_no_null, columns=columns,categorical=categorical)
seer_tableone.to_csv('seer_tableone.csv')